In [0]:
%pip install databricks-feature_engineering

In [0]:
%pip install lightgbm

In [0]:
%restart_python

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import json
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import joblib
from datetime import datetime, date
import mlflow
import mlflow.lightgbm
from sklearn.metrics import classification_report,roc_auc_score,f1_score
from mlflow.models.signature import infer_signature
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from sklearn.preprocessing import LabelEncoder
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from pyspark.sql.functions import col

In [0]:
base_4 = spark.table('ispl_databricks.model_logs.bd_500_features_sample_training')

In [0]:
model = joblib.load('/Workspace/Shared/ff_bd/model_artifacts/top50model.pkl')

In [0]:
base_4 = spark.table('ispl_databricks.model_logs.bd_500_features_sample_training')

In [0]:
spark_label =  base_4.select(col('loan_id'),col('target'))

In [0]:
fe = FeatureEngineeringClient()

In [0]:
training_set = fe.create_training_set(
    df=spark_label,
    feature_lookups=[
        FeatureLookup(
            table_name="ispl_databricks.model_logs.bd_final_feature_stores",
            lookup_key="loan_id"
        )
    ],
    label="target"
)


In [0]:
train_pd = training_set.load_df().toPandas()
train_pd = train_pd.dropna()
train_x  = train_pd.drop(['loan_id','target'], axis=1)
train_y = train_pd['target']

In [0]:
input_X = train_x.iloc[[0]]

In [0]:
trained_feature_names = model.feature_name_

In [0]:
len(trained_feature_names)

In [0]:
feature_json = {'features': trained_feature_names}

In [0]:
with open('/Workspace/Shared/ff_bd/model_artifacts/model_features.json','w') as f:
    json.dump(feature_json, f, ensure_ascii=False, indent=4)

In [0]:
input_X_aligned = input_X[trained_feature_names]

In [0]:
output = model.predict_proba(input_X_aligned)

In [0]:
signature = infer_signature(input_X_aligned, output)

In [0]:
with open('/Workspace/Shared/ff_bd/model_artifacts/model_features.json') as file:
    data = json.load(file)

In [0]:
with open('/Workspace/Shared/ff_bd/model_artifacts/model_metric.json') as file:
    metric = json.load(file)

In [0]:
data

In [0]:
class mlwrapper(mlflow.pyfunc.PythonModel):
    def load_context(self,context):
        self.model = joblib.load(context.artifacts['model_artifacts']+'/top50model.pkl')
        with open(context.artifacts['model_artifacts']+'/model_features.json', 'r') as file:
            data = json.load(file)
        self.fc = data['features']
        print(self.fc)
        
    def predict(self,context,model_input):
        df = model_input[self.fc]
        return self.model.predict_proba(df)

In [0]:
with mlflow.start_run():
    mlflow.log_metric("test_accuracy", metric['accuracy'])

    

    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=mlwrapper(),
        artifacts={"model_artifacts": "/Workspace/Shared/ff_bd/model_artifacts"},
        registered_model_name="ispl_databricks.model_logs.final_bd_model",
        signature=signature
    )

In [0]:

from mlflow.tracking import MlflowClient
client = MlflowClient()
versions = client.search_model_versions("name = 'ispl_databricks.model_logs.final_bd_model'")

latest_version = sorted(versions, key=lambda v: int(v.version))[-1].version

In [0]:
model_uri = f"models:/ispl_databricks.model_logs.final_bd_model/{latest_version}"

In [0]:
model = mlflow.pyfunc.load_model(model_uri=model_uri)

In [0]:
model.predict(input_X_aligned)